#  BCS Computational Tutorial: LSTM Sentiment Classifier

We'll be building a tweet sentiment classifier by representing tweets as *sequences* of words, with a LSTM.

First import some things and go over the basics of TensorFlow:

In [1]:
import math
import pickle as p
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import utils
import json
import time

Then, check which devices are available to tensorflow, and select the device you want to use:

In [2]:
# Set logging output to display during runtime. (n.b. doesn't work in Jupyter)
#tf.debugging.set_log_device_placement(True)
# Print device list and detailed info about devices.
print('Device List:\n', tf.config.experimental.list_physical_devices(), '\n\n')
print('Device Details:\n', device_lib.list_local_devices(), '\n\n')
# Select device to use, set-up computation graph, and run computation graph.
with tf.device('/device:GPU:1'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
    sess = tf.Session()
    options = tf.RunOptions(output_partition_graphs=True)
    metadata = tf.RunMetadata()
    c_val = sess.run(c, options=options, run_metadata=metadata)
    last_node = metadata.partition_graphs[0].node[-1]
    print('Runtime Metadata Last Node Details:\n', last_node.name, ' ', last_node.device)

Device List:
 [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')] 


Device Details:
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2695590965871351023
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6614766059
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14129835957610308328
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:2d:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 6663627080
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8645950860784130695
physical_device_desc: "device: 1, name: GeForce GTX 1080, pci bus id: 0000:23:00.0, compute capability: 6.1"
] 


Runtime Metadata Last Node Details:
 MatMul/_0__cf__0   /job:localhost/replica:0/task:0/device:GPU:1


# Intro to TensorFlow

## What is a Computation Graph?

Everything in TensorFlow comes down to building a computation graph. What is a computation graph? Its just a series of math operations that occur in some order. Here is an example of a simple computation graph:

<img src="computation-graph.png">

This graph takes 2 inputs, (a, b) and computes an output (e). Each node in the graph is an operation that takes some input, does some computation, and passes its output to another node.

We could make this computation graph in TensorFlow in the following way:

In [3]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.add(a, b)
d = tf.subtract(b, 1)
e = tf.multiply(c, d)

Tensorflow uses tf.placeholder to handle inputs to the model. This is like making a reservation at a restaurant. The restaurant reserves a spot for 5 people, but you are free to fill those seats with any set of friends you want to. tf.placeholder lets you specify that some input will be coming in, of some shape and some type. Only when you run the computation graph do you actually provide the values of this input data. You would run this simple computation graph like this:

In [4]:
with tf.Session() as session:
    a_data, b_data = 3.0, 6.0
    feed_dict = {a: a_data, b: b_data}
    output = session.run([e], feed_dict=feed_dict)
    print(output) # 45.0

[45.0]


We use feed_dict to pass in the actual input data into the graph. We use session.run to get the output from the c operation in the graph. Since e is at the end of the graph, this ends up running the entire graph and returning the number 45 - cool!

## Neural Networks in Tensorflow

We can define neural networks in TensorFlow using computation graphs. Here is an example, very simple neural network (just 1 perceptron):

<img src="computation-graph-2.png">

This graph takes an input, (x) and computes an output (out). It does it with what we learned in class, `out = sigmoid(W*x+b)`.

We could make this computation graph in TensorFlow in the following way:

In [5]:
n_input_nodes = 2
n_output_nodes = 1
x = tf.placeholder(tf.float32, (None, n_input_nodes))
W = tf.Variable(tf.ones((n_input_nodes, n_output_nodes)), dtype=tf.float32)
b = tf.Variable(tf.zeros(n_output_nodes), dtype=tf.float32)

'''TODO: Define the operation for z (use tf.matmul to multiply W and x).'''
z = tf.matmul(x,W) + b

'''TODO: Define the operation for out (use tf.sigmoid).'''
out = tf.sigmoid(z)

To run this graph, we again use session.run() and feed in our input via feed_dict.

In [6]:
test_input = [[0.5, 0.5]]
with tf.Session() as session:
    tf.global_variables_initializer().run(session=session)
    feed_dict = {x: test_input}
    output = session.run([out], feed_dict=feed_dict)
    print(output[0]) # This should output 0.73105. If not, double-check your code above

[[0.7310586]]


#  LSTMs for Tweet Sentiment Classification
Our model will be like this:

![alt-text](lab2-diagram.png)

We feed words one by one into LSTM layers.  After feeding in all the words, we take the final state of the LSTM and run it thorugh one fully connected layer to multiply it by a final set of weights. We specificy that this fully connected layer should have a single output, which, when sigmoid-ed, is the probability that the tweet is positive!

## Step 1: Set up our Model Parameters

Similarly to the last lab, we'll be training using batches. Our hidden layer will have 100 units, and we have 7597 words in the vocabulary. 

In [7]:
# set variables
tweet_size = 20
hidden_size = 100
vocab_size = 7597

# this just makes sure that all our following operations will be placed in the right graph.
tf.reset_default_graph()

# create a session variable that we can run later.
session = tf.Session()

## Step 2: Create Placeholders

We need to create placeholders for variable data that we will feed in ourselves (aka our tweets). Placeholders allow us to incorporate this data into the graph even though we don't know what it is yet.

In [8]:
# the placeholder for tweets has first dimension batch_size for each tweet in a batch,
# second dimension tweet_size for each word in the tweet, and third dimension vocab_size
# since each word itself is represented by a one-hot vector of size vocab_size.
# Note that we use 'None' instead of batch_size for the first dimsension.  This allows us 
# to deal with variable batch sizes
tweets = tf.placeholder(tf.float32, [None, tweet_size, vocab_size])

'''TODO: create a placeholder for the labels (our predictions).  
   This should be a 1D vector with size = None, 
   since we are predicting one value for each tweet in the batch,
   but we want to be able to deal with variable batch sizes.''';
labels = tf.placeholder(tf.float32, [None])

## Step 3: Build the LSTM Layers

We want to feed the input sequence, word by word, into an LSTM layer, or multiple LSTM layers (we could also call this an LSTM **encoder**).  At each "timestep", we feed in the next word, and the LSTM updates its cell state. The final LSTM cell state can then be fed through a final classification layer(s) to get our sentiment prediction.  

Now let's make our LSTM layer. The steps for this are: 
1. Create a LSTM Cell using [tf.contrib.rnn.LSTMCell](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell)

2. Wrap a couple of these cells in [tf.nn.rnn_cell.MultiRNNCell](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell) to create a multiple LSTM layers.

2. Define the operation to run these layers with [dynamic_rnn](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [9]:
'''TODO: create 2 LSTM Cells using BasicLSTMCell.  Note that this creates a *layer* of LSTM
   cells, not just a single one.''';
lstm_cell_1 = tf.contrib.rnn.LSTMCell(hidden_size)
lstm_cell_2 = tf.contrib.rnn.LSTMCell(hidden_size)

'''TODO: create two LSTM layers by wrapping two instances of 
   lstm_cell from above in tf.contrib.rnn_cell.MultiRNNCell. Note that
   you can use multiple cells by doing [cell1, cell2]. Also note
   that you should use state_is_tuple=True as an argument.  This will allow
   us to access the part of the cell state that we need later on.''';
multi_lstm_cells = multi_lstm_cells = tf.contrib.rnn.MultiRNNCell(cells=[lstm_cell_1, lstm_cell_2] , state_is_tuple=True)

'''TODO: define the operation to create the RNN graph across time.  
   tf.nn.dynamic_rnn dynamically constructs the graph when it is executed,
   and returns the final cell state.''';
outputs, final_state = tf.nn.dynamic_rnn(multi_lstm_cells, tweets, dtype=tf.float32)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Step 4: Classification Layer

Now we have the final state of the LSTM layers after feeding in the tweet word by word. We can take this final state and feed it into a simple classfication layer that takes the cell state, multiplies it by some weight matrix (with bias) and outputs a single value corresponding to whether it thinks the tweet is overall positive or not.  

In [10]:
## We define this function that creates a weight matrix + bias parameter
## and uses them to do a matrix multiplication.
def linear(input_, output_size, name, init_bias=0.0):
    shape = input_.get_shape().as_list()
    with tf.variable_scope(name):
        W = tf.get_variable("weights", [shape[-1], output_size], tf.float32, tf.random_normal_initializer(stddev=1.0 / math.sqrt(shape[-1])))
    if init_bias is None:
        return tf.matmul(input_, W)
    with tf.variable_scope(name):
        b = tf.get_variable("bias", [output_size], initializer=tf.constant_initializer(init_bias))
    return tf.matmul(input_, W) + b

In [11]:
'''TODO: pass the final state into this linear function to multiply it 
   by the weights and add bias to get our output.
   
   {Quick note that we need to feed in final_state[-1][-1] into linear since 
   final_state is actually a tuple consisting of the cell state 
   (used internally for the cell to keep track of things) 
   as well as the hidden state (the output of the cell), and one of these 
   tuples for each layer. We want the hidden state for the last layer, so we use 
   final_state[-1][-1]}''';

sentiment = linear(final_state[-1][-1], 1, name="output")

## Step 5: Define Loss

Now we define a loss function that we'll use to determine the difference between what we predicted and what's actually correct.  We'll want to use cross entropy, since we can take into account what probability the model gave to the a tweet being positive.

The output we just got from the linear classification layer is called a 'logit' -- the raw value before transforming it into a probability between 0 and 1.  We can feed these logits to  [`tf.nn.sigmoid_cross_entropy_with_logits`](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits), which will take the sigmoid of these logits (making them between 0 and 1) and then calculate the cross-entropy with the ground truth labels.

In [12]:
sentiment = tf.squeeze(sentiment, [1])

'''TODO: define our loss function.  
   We will use tf.nn.sigmoid_cross_entropy_with_logits, which will compare our 
   sigmoid-ed prediction (sentiment from above) to the ground truth (labels).''';

loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=sentiment, labels=labels)

# our loss with sigmoid_cross_entropy_with_logits gives us a loss for each 
# example in the batch. We take the mean of all these losses.
loss = tf.reduce_mean(loss)

# to get actual results like 'positive' or 'negative' , 
# we round the prediction probability to 0 or 1.
prediction = tf.to_float(tf.greater_equal(sentiment, 0.5))

# calculate the error based on which predictions were actually correct.
pred_err = tf.to_float(tf.not_equal(prediction, labels))
pred_err = tf.reduce_sum(pred_err)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.


## Step 6: Train

Now we define the operation that actually changes the weights by minimizing the loss.  

[`tf.train.AdamOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) is just a gradient descent algorithm that uses a variable learning rate to converge faster and more effectively.

We want to specify this optimizer and then call the [minimize](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer#minimize) function, the optimizer knows it wants to minimize the loss we defined above.

In [13]:
'''Define the operation that specifies the AdamOptimizer and tells
   it to minimize the loss.''';
optimizer = tf.train.AdamOptimizer().minimize(loss)

## Step 7: Run Session!

Now that we've made all the variable and operations in our graph, we can load the data, feed it in...

In [22]:
# initialize any variables
tf.global_variables_initializer().run(session=session)

# load our data and separate it into tweets and labels
train_data = json.load(open('data/trainTweets_preprocessed.json', 'r'))
train_data = list(map(lambda row:(np.array(row[0],dtype=np.int32),str(row[1])),train_data))
train_tweets = np.array([t[0] for t in train_data])
train_labels = np.array([int(t[1]) for t in train_data])

test_data = json.load(open('data/testTweets_preprocessed.json', 'r'))
test_data = list(map(lambda row:(np.array(row[0],dtype=np.int32),str(row[1])),test_data))
# we are just taking the first 1000 things from the test set for faster evaluation
test_data = test_data[0:1000] 
test_tweets = np.array([t[0] for t in test_data])
one_hot_test_tweets = utils.one_hot(test_tweets, vocab_size)
test_labels = np.array([int(t[1]) for t in test_data])

# we'll train with batches of size 32. This means that we run 
# our model on 32 examples and then do gradient descent based on the loss
# over those 32 examples.
batch_size = 32
num_steps = np.int(len(train_data) / batch_size)
num_epochs = 5

And run the training!

In [23]:
t0 = time.perf_counter()
for epoch in range(num_epochs):
    for step in range(num_steps):
        # get data for a batch
        offset = (step * batch_size) % (len(train_data) - batch_size)
        batch_tweets = utils.one_hot(train_tweets[offset : (offset + batch_size)], vocab_size)
        batch_labels = train_labels[offset : (offset + batch_size)]

        # put this data into a dictionary that we feed in when we run 
        # the graph. This data fills in the placeholders we made in the graph.
        data = {tweets: batch_tweets, labels: batch_labels}

        # run the 'optimizer', 'loss', and 'pred_err' operations in the graph
        _, loss_value_train, error_value_train = session.run(
          [optimizer, loss, pred_err], feed_dict=data)
    
    # print stuff every epoch to see how we are doing
    print("Time spent in last epoch: ", time.perf_counter() - t0)
    print("Train loss at epoch", epoch, ":", loss_value_train)
    print("Train error: %.3f%%" % error_value_train)

    # get test evaluation
    test_loss = []
    test_error = []
    for batch_num in range(int(len(test_data)/batch_size)):
        test_offset = (batch_num * batch_size) % (len(test_data) - batch_size)
        test_batch_tweets = one_hot_test_tweets[test_offset : (test_offset + batch_size)]
        test_batch_labels = test_labels[test_offset : (test_offset + batch_size)]
        data_testing = {tweets: test_batch_tweets, labels: test_batch_labels}
        loss_value_test, error_value_test = session.run([loss, pred_err], feed_dict=data_testing)
        test_loss.append(loss_value_test)
        test_error.append(error_value_test)

    print("Test loss: %.3f" % np.mean(test_loss))
    print("Test error: %.3f%%" % np.mean(test_error))
print("\nTime spent training: ", time.perf_counter() - t0)

Time spent in last epoch:  80.31361320000002
Train loss at epoch 0 : 0.43616664
Train error: 8.000%
Test loss: 0.513
Test error: 8.774%
Time spent in last epoch:  161.97296820000003
Train loss at epoch 1 : 0.35408577
Train error: 4.000%
Test loss: 0.517
Test error: 8.645%
Time spent in last epoch:  243.72728009999997
Train loss at epoch 2 : 0.25547802
Train error: 4.000%
Test loss: 0.586
Test error: 8.516%
Time spent in last epoch:  325.6937339
Train loss at epoch 3 : 0.23870307
Train error: 5.000%
Test loss: 0.724
Test error: 9.323%
Time spent in last epoch:  406.9539039
Train loss at epoch 4 : 0.15966126
Train error: 3.000%
Test loss: 0.727
Test error: 9.710%
Time spent training:  407.63247879999994


Running this code, you’ll see the network train and output its performance as it learns. 

## Concluding Thoughts
This was a brief introduction into TensorFlow. There is so, so much more to learn and explore, but hopefully this has given you some base knowledge to expand upon. As an additional exercise, you can see what you can do with this code to improve the performance. Ideas include: randomizing mini-batches, making the network deeper, using word embeddings (see below) rather than bag-of-words, trying different optimizers (like Adam), different weight initializations.

#### More on Word Embeddings

In this lab we used one-hot vectors to represent words in tweets.  Word Embeddings are a more meaningful representation.  The basic idea is we represent a word with a vector $\phi$ by the context the word appears in. We do this by training a neural network to predict the context of words across a large training set. The weights of that neural network can then be thought of as a dense and useful representation that captures context. This is useful because now our representations of words captures actual semantic similarity.

Word Embeddings capture all kinds of useful semantic relationships. For example, one cool emergent property is $ \phi(king) - \phi(queen) = \phi(man) - \phi(woman)$. To learn more about the magic behind word embeddings we recommend Chris Olah's [blog post](https://colah.github.io/posts/2014-07-NLP-RNNs-Representations/). A common tool for generating Word Embeddings is word2vec.

## Solutions


In [ ]:
import math
import pickle as p
import tensorflow as tf
import numpy as np
import utils 

# set variables
tweet_size = 20
hidden_size = 100
vocab_size = 7597
batch_size = 64

# this just makes sure that all our following operations will be placed in the right graph.
tf.reset_default_graph()

# create a session variable that we can run later.
session = tf.Session()

# make placeholders for data we'll feed in
tweets = tf.placeholder(tf.float32, [None, tweet_size, vocab_size])
labels = tf.placeholder(tf.float32, [None])

# make the lstm cells, and wrap them in MultiRNNCell for multiple layers
lstm_cell_1 = tf.contrib.rnn.LSTMCell(hidden_size)
lstm_cell_2 = tf.contrib.rnn.LSTMCell(hidden_size)
multi_lstm_cells = tf.contrib.rnn.MultiRNNCell(cells=[lstm_cell_1, lstm_cell_2] , state_is_tuple=True)

# define the op that runs the LSTM, across time, on the data
outputs, final_state = tf.nn.dynamic_rnn(multi_lstm_cells, tweets, dtype=tf.float32)

# a useful function that takes an input and what size we want the output 
# to be, and multiples the input by a weight matrix plus bias (also creating
# these variables)
def linear(input_, output_size, name, init_bias=0.0):
    shape = input_.get_shape().as_list()
    with tf.variable_scope(name):
        W = tf.get_variable("weight_matrix", [shape[-1], output_size], tf.float32, tf.random_normal_initializer(stddev=1.0 / math.sqrt(shape[-1])))
    if init_bias is None:
        return tf.matmul(input_, W)
    with tf.variable_scope(name):
        b = tf.get_variable("bias", [output_size], initializer=tf.constant_initializer(init_bias))
    return tf.matmul(input_, W) + b

# define that our final sentiment logit is a linear function of the final state 
# of the LSTM
sentiment = linear(final_state[-1][-1], 1, name="output")


sentiment = tf.squeeze(sentiment, [1])

# define cross entropy loss function
loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=sentiment, labels=labels)
loss = tf.reduce_mean(loss)

# round our actual probabilities to compute error
prob = tf.nn.sigmoid(sentiment)
prediction = tf.to_float(tf.greater_equal(prob, 0.5))
pred_err = tf.to_float(tf.not_equal(prediction, labels))
pred_err = tf.reduce_sum(pred_err)

# define our optimizer to minimize the loss
optimizer = tf.train.AdamOptimizer().minimize(loss)

# initialize any variables
tf.global_variables_initializer().run(session=session)

# load our data and separate it into tweets and labels
train_data = json.load(open('data/trainTweets_preprocessed.json', 'r'))
train_data = list(map(lambda row:(np.array(row[0],dtype=np.int32),str(row[1])),train_data))

train_tweets = np.array([t[0] for t in train_data])
train_labels = np.array([int(t[1]) for t in train_data])

test_data = json.load(open('data/testTweets_preprocessed.json', 'r'))
test_data = map(lambda row:(np.array(row[0],dtype=np.int32),str(row[1])),test_data)
# we are just taking the first 1000 things from the test set for faster evaluation
test_data = test_data[0:1000] 
test_tweets = np.array([t[0] for t in test_data])
one_hot_test_tweets = utils.one_hot(test_tweets, vocab_size)
test_labels = np.array([int(t[1]) for t in test_data])

# we'll train with batches of size 128.  This means that we run 
# our model on 128 examples and then do gradient descent based on the loss
# over those 128 examples.
num_steps = 1000

for step in range(num_steps):
    # get data for a batch
    offset = (step * batch_size) % (len(train_data) - batch_size)
    batch_tweets = utils.one_hot(train_tweets[offset : (offset + batch_size)], vocab_size)
    batch_labels = train_labels[offset : (offset + batch_size)]
    
    # put this data into a dictionary that we feed in when we run 
    # the graph.  this data fills in the placeholders we made in the graph.
    data = {tweets: batch_tweets, labels: batch_labels}
    
    # run the 'optimizer', 'loss', and 'pred_err' operations in the graph
    _, loss_value_train, error_value_train = session.run(
      [optimizer, loss, pred_err], feed_dict=data)
    
    # print stuff every 50 steps to see how we are doing
    if (step % 50 == 0):
        print("Minibatch train loss at step", step, ":", loss_value_train)
        print("Minibatch train error: %.3f%%" % error_value_train)
        
        # get test evaluation
        test_loss = []
        test_error = []
        for batch_num in range(int(len(test_data)/batch_size)):
            test_offset = (batch_num * batch_size) % (len(test_data) - batch_size)
            test_batch_tweets = one_hot_test_tweets[test_offset : (test_offset + batch_size)]
            test_batch_labels = test_labels[test_offset : (test_offset + batch_size)]
            data_testing = {tweets: test_batch_tweets, labels: test_batch_labels}
            loss_value_test, error_value_test = session.run([loss, pred_err], feed_dict=data_testing)
            test_loss.append(loss_value_test)
            test_error.append(error_value_test)
        
        print("Test loss: %.3f" % np.mean(test_loss))
        print("Test error: %.3f%%" % np.mean(test_error))